## Daily Candles


looking at the candle formations of the daily candle to take trades, this will ideally act as an entry signal for long term holds

### Backtesting symbols

In [82]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from talib import CDLENGULFING, CDLMORNINGSTAR, CDLSHOOTINGSTAR 
from backtesting import Backtest, Strategy

# Initialize MetaTrader 5
mt5.initialize()

# Function to get data from MetaTrader 5
def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df = df[['time', 'open', 'high', 'low', 'close']]
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

# Functions to add candlestick patterns and SMA to the DataFrame
def get_engulfing(symbol):
    symbol['engulfing'] = CDLENGULFING(symbol['Open'], symbol['High'], symbol['Low'], symbol['Close'])
    return symbol

def get_morningstar(symbol):
    symbol['morningstar'] = CDLMORNINGSTAR(symbol['Open'], symbol['High'], symbol['Low'], symbol['Close'])
    return symbol 

def get_shootingstar(symbol):
    symbol['shootingstar'] = CDLSHOOTINGSTAR(symbol['Open'], symbol['High'], symbol['Low'], symbol['Close'])
    return symbol  

def get_sma(symbol, period):
    symbol['sma50'] = symbol['Close'].rolling(period).mean()
    return symbol

def SIGNAL(data):
    return data['engulfing']

# Define the Candlesticks strategy
class Candlesticks(Strategy):
    initsize = 100.0
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL, self.data.df)
        
    def next(self):
        super().next()
        if self.data.Close[-1] < self.data.sma50[-1] and self.signal1[-1] == 100:
            sl1 = self.data.Close[-1] * 0.99
            tp1 = self.data.Close[-1] * 1.03
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        if self.data.Close[-1] > self.data.sma50[-1] and self.signal1[-1] == -100:
            sl1 = self.data.Close[-1] * 1.01
            tp1 = self.data.Close[-1] * 0.97
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

if __name__ == '__main__':
    # Settings
    symbols = ['EURUSD', 'GBPUSD', 'GBPJPY',  'USDCHF', 'USDCAD', 
               'NZDUSD', 'USDJPY', 'AUDUSD', 'CADCHF', 'AUDNZD', 
               'AUDCAD', 'AUDCHF', 'EURGBP', 'CADJPY', 'GBPAUD']
    timeframe = mt5.TIMEFRAME_D1
    start_date = datetime(2022, 1, 1)
    end_time = datetime.now()

    for symbol in symbols:
        # Get data and analyze
        data = get_data(symbol, timeframe, start_date, end_time)
        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
        data = get_engulfing(data)
        data = get_morningstar(data)
        data = get_shootingstar(data)
        data = get_sma(data, 20)

        # Conduct backtest
        bt = Backtest(data, Candlesticks, cash=10000, exclusive_orders=True, margin=1/100)

        # Outputs
        output = bt.run()
        print(f"Symbol: {symbol}")
        print(output[['Sharpe Ratio', 'Return [%]']])
        print('-' * 50)


Symbol: EURUSD
Sharpe Ratio    0.601797
Return [%]      0.073947
dtype: object
--------------------------------------------------
Symbol: GBPUSD
Sharpe Ratio         0.0
Return [%]     -0.198325
dtype: object
--------------------------------------------------
Symbol: GBPJPY
Sharpe Ratio    0.05092
Return [%]      1.04032
dtype: object
--------------------------------------------------
Symbol: USDCHF
Sharpe Ratio     0.38674
Return [%]      0.036421
dtype: object
--------------------------------------------------
Symbol: USDCAD
Sharpe Ratio         0.0
Return [%]     -0.039211
dtype: object
--------------------------------------------------
Symbol: NZDUSD
Sharpe Ratio         0.0
Return [%]     -0.035518
dtype: object
--------------------------------------------------
Symbol: USDJPY
Sharpe Ratio        0.0
Return [%]     -0.68065
dtype: object
--------------------------------------------------
Symbol: AUDUSD
Sharpe Ratio    0.425366
Return [%]      0.040236
dtype: object
---------------

### Plotting to see the trades

In [84]:

# Function to get data from MetaTrader 5
def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df = df[['time', 'open', 'high', 'low', 'close']]
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

# Functions to add candlestick patterns and SMA to the DataFrame
def get_engulfing(symbol):
    symbol['engulfing'] = CDLENGULFING(symbol['Open'], symbol['High'], symbol['Low'], symbol['Close'])
    return symbol

def get_morningstar(symbol):
    symbol['morningstar'] = CDLMORNINGSTAR(symbol['Open'], symbol['High'], symbol['Low'], symbol['Close'])
    return symbol 

def get_shootingstar(symbol):
    symbol['shootingstar'] = CDLSHOOTINGSTAR(symbol['Open'], symbol['High'], symbol['Low'], symbol['Close'])
    return symbol  

def get_sma(symbol, period):
    symbol['sma50'] = symbol['Close'].rolling(period).mean()
    return symbol

def SIGNAL(data):
    return data['engulfing']

# Define the Candlesticks strategy
class Candlesticks(Strategy):
    initsize = 100.0
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL, self.data.df)
        
    def next(self):
        super().next()
        if self.data.Close[-1] < self.data.sma50[-1] and self.signal1[-1] == 100:
            sl1 = self.data.Close[-1] * 0.99
            tp1 = self.data.Close[-1] * 1.03
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        if self.data.Close[-1] > self.data.sma50[-1] and self.signal1[-1] == -100:
            sl1 = self.data.Close[-1] * 1.01
            tp1 = self.data.Close[-1] * 0.97
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

if __name__ == '__main__':
    # Settings
    symbols = ['EURGBP']
    timeframe = mt5.TIMEFRAME_D1
    start_date = datetime(2022, 1, 1)
    end_time = datetime.now()

    for symbol in symbols:
        # Get data and analyze
        data = get_data(symbol, timeframe, start_date, end_time)
        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
        data = get_engulfing(data)
        data = get_morningstar(data)
        data = get_shootingstar(data)
        data = get_sma(data, 20)

        # Conduct backtest
        bt = Backtest(data, Candlesticks, cash=10000, exclusive_orders=True, margin=1/100)

        # Outputs
        output = bt.run()
        print(f"Symbol: {symbol}")
        print(output)
        print('-' * 50)
        bt.plot()


Symbol: EURGBP
Start                     2022-01-01 00:00:00
End                       2024-05-21 00:00:00
Duration                    871 days 00:00:00
Exposure Time [%]                   58.024691
Equity Final [$]                  10007.20496
Equity Peak [$]                   10007.30096
Return [%]                            0.07205
Buy & Hold Return [%]                 1.53495
Return (Ann.) [%]                      0.0249
Volatility (Ann.) [%]                0.029184
Sharpe Ratio                          0.85322
Sortino Ratio                        1.334272
Calmar Ratio                         0.848754
Max. Drawdown [%]                   -0.029337
Avg. Drawdown [%]                   -0.005644
Max. Drawdown Duration      301 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   25
Win Rate [%]                             68.0
Best Trade [%]                            3.0
Worst Trade [%]                     -1.008429
Avg. Trade [%]     